# 트리의 앙상블
* 앙상블 학습법은 더 좋은 예측 성능을 얻기위해 다수의 학습 알고리즘을 사용하는 방법이다

## 랜덤포레스트
* 결정트리를 랜덤하게 만들어 결정트리(나무)의 숲을 만든다
* 랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데 한 샘플이 중복되어 추출될 수도 있다
* 중복되어서 만들어진 샘플을 부트스트랩 샘플 이라고 한다
* 부트스트랩 샘플은 훈련세트의 크기와 같게 만들어서 각각의 부트스트랩 샘플별로 결정트리훈련을 수행한다
* RandomForestClassifier는 전체 특성개수의 제곱근 만큼의 특성을 선택하여 사용한다. 
* RandomForestRegressor는 전체 특성을 사용한다
* 랜덤포레스트는 기본적으로 100개의 결정트리를 훈련합니다 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine/wine.csv')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

X_train, X_test, y_train, y_test = \
train_test_split(data, target, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


* 기본적으로 100개의 결정트리를 사용하므로 n_jobs를 -1 로 지정하여 모든 CPU 코어를 사용하는 것이 좋다
* cross_validate() 함수에서 return_train_score 매개변수를 True로 지정하면 훈련세트에 대한 점수도 검증세트 점수와 같이 반환합니다.

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, X_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(X_train, y_train)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


* 부트스트랩에 샘플에 포함되지 않고 남는 샘플을 OOB(out of bag) 샘플이라고 한다
* OOB 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정트리를 평가할 수 있다
* 교차검증세트에서 얻은 점수가 비슷한 결과가 나온다  0.8905151032797809 / 0.8934000384837406

In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(X_train, y_train)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리
* 랜덤 포레스트와 비슷하게 동작한다 차이점은 부트스트랩 샘플을 사용하지 않는다
* 엑스트라 트리가 사용하는 결정트리가 splitter='random' 인 결정트리이다

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, X_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [7]:
et.fit(X_train, y_train)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레디언트 부스팅
* 그레디언트 부스팅은 깊이가 얕은 결정트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이다
* Scikit-Learn의 GradientBoostingClassifier는 깊이가 3인 결정트리를 100개 사용한다
 - 깊이가 얕은 결정트리를 사용하기 때문에 과대적합에 강하고, 높은 일반화 성능을 기대할 수 있다
* 경사하강법을 사용하여 트리를 앙상블에 추가한다 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, X_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

* n_estimators는 결정트리의 갯수이다. 
* 결정트리 갯수를 5배나 늘렸지만 과대적합을 잘 억제한다

In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, X_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

* 그레이언트 부스팅이 랜덤 포레스트 보다 당도(sugar)에 좀 더 집중한다

In [ ]:
gb.fit(X_train, y_train)
print(gb.feature_importances_)

## 히스토그램 기반 부스팅
* 그레디언트 부스팅이 랜덤 포레스트 보다 좀 더 높은 성능을 내지만 순서대로 트리를 추가하지 때문에 속도가 느리며, n_jobs 매개변수도 없다
* 그레디언트 부스팅의 속도와 성능을 개선한 모델이 히스토그램기반 부스팅이다
* 히스토그램기반 부스팅은 입력 특성(feature)를 256개의 구간으로 나누기 때문에 노드를 분할할때 최적의 분할을 매우 빠르게 찾을 수 있다
* HistGradientBoostingClassifier 아직 테스트 과정에 있기 때문에 이 클래스를 사용하려면 sklearn.experimental 패키지 아래에 있는 enable_hist_gradient_boosting 모듈을 같이 import 해야 한다


In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, X_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

* Permutation Importance는 모델을 학습시킨 뒤, 특정 feature의 데이터를 shuffle 했을 때, 검증 데이터 셋에 대한 예측성능을 확인하고 feature importance를 계산한다.


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, X_train, y_train, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

In [ ]:
hgb.score(X_test, y_test)

#### XGBoost

In [ ]:
# from xgboost import XGBClassifier

# xgb = XGBClassifier(tree_method='hist', random_state=42)
# scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

# print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#### LightGBM

In [ ]:
# from lightgbm import LGBMClassifier

# lgb = LGBMClassifier(random_state=42)
# scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

# print(np.mean(scores['train_score']), np.mean(scores['test_score']))